These finetuning steps are based on the tutorial linked here: https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

C:\Users\sjkov\miniforge3\envs\ml\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [5]:
df = scraper.hf_dataframe()
    
qa = df[["question","answer"]]
    
qa_clean = qa.map(scraper.cleaner)
        
#occ_dict = scraper.occ_dict_maker(qa_clean,'question')
    
basket, word_dict  = scraper.create_basket('word_occurences.csv',20,1000)

ans_list, ans_dict  = scraper.create_basket('Answer_occurences.csv',10,20000)

    #scraper.find_good_questions(qa_clean, ans_list, 'good_questions.csv')

good_qs = scraper.retrieve_good_qs('good_questions.csv')

    #print(good_qs)
    
    #print(datetime.now())
count = 1
X_list = []
y_list = []
    
    #print (len(basket))
    #print(len(ans_list))

num_qs = 20000

In [7]:
#goes through each question and makes an X vector using one hot encoding and a Y vector, which has a number represent the answer
for i in good_qs[0:num_qs]:
        
    tempVector = np.zeros(len(basket)).tolist()
    for word in qa_clean.loc[int(i),"question"].split():
            #print(word)
        if word in basket:
            tempVector[basket.index(word)] = 1
                
    X_list.append(tempVector)
    y_list.append(ans_list.index(qa_clean.loc[int(i),"answer"]))
        #print(count)
        #count+=1
       
X = np.array(X_list)
y = np.array(y_list)
    
    #for ans in y:
        #print(ans_list[ans])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=33)

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#finish preprocessing here

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


In [ ]:
#figure out tensorboard setup here

In [ ]:
#bert finetuning here
#experiment with different parameters here
args = TrainingArguments(output_dir=model_output_dir, evaluation_strategy="steps", eval_steps=50, logging_strategy="steps", logging_steps=50,
                         save_strategy="steps", save_steps = 200, learning rate = 5e-5, num_train_epochs=1, load_best_model_at_end=True, metric_for_best_model="accuracy",
                         report_to="tensorboard")

In [ ]:
#pass to trainer object, in progress

In [ ]:
#compute the metrics (accuracy)